En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
file_path = "farmers-protest-tweets-2021-2-4.json"

# ETL

Se crea un proceso de Extracción, transformación y carga del archivo, lo que permitirá procesar el archivo directamente en Bigquery, parte de Google Cloud Bigquery (GCP), esto permitirá optimizar la velocidad de cálculo y la utilización de SQL para desarrollar el análisis.

### Credenciales

Este desarrollo se realizará utilizando Google Cloud, por lo que se crea un proyecto en GCP llamado "project-latam-challenge". En este proyecto se crea una service account llamada "sa-etl-latam-challenge" que será utilizada para realizar la carga de datos.

In [2]:
import os

# Ruta a archivo de credenciales JSON de Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../credentials/project-latam-challenge-749ce1a96052.json"


### Carga en dataframe

Se carga en un dataframe el [archivo](https://drive.google.com/file/d/1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis/view?usp=sharing) json declarado como parte del challenge.

In [3]:
import pandas as pd

# Leer el archivo CSV en un DataFrame
df = pd.read_json(file_path,lines=True)

# Mostrar las primeras filas del DataFrame
df.head()

url  \
0  https://twitter.com/ArjunSinghPanam/status/136...   
1  https://twitter.com/PrdeepNain/status/13645062...   
2  https://twitter.com/parmarmaninder/status/1364...   
3  https://twitter.com/anmoldhaliwal/status/13645...   
4  https://twitter.com/KotiaPreet/status/13645061...   

                       date  \
0 2021-02-24 09:23:35+00:00   
1 2021-02-24 09:23:32+00:00   
2 2021-02-24 09:23:22+00:00   
3 2021-02-24 09:23:16+00:00   
4 2021-02-24 09:23:10+00:00   

                                             content  \
0  The world progresses while the Indian police a...   
1  #FarmersProtest \n#ModiIgnoringFarmersDeaths \...   
2  ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...   
3  @ReallySwara @rohini_sgh watch full video here...   
4  #KisanEktaMorcha #FarmersProtest #NoFarmersNoF...   

                                     renderedContent                   id  \
0  The world progresses while the Indian police a...  1364506249291784198   
1  #FarmersProtest \n#ModiIgnoringFarmersDeaths \...  1364506237451313155   
2  ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...  1364506195453767680   
3  @ReallySwara @rohini_sgh watch full video here...  1364506167226032128   
4  #KisanEktaMorcha #FarmersProtest #NoFarmersNoF...  1364506144002088963   

                                                user  \
0  {'username': 'ArjunSinghPanam', 'displayname':...   
1  {'username': 'PrdeepNain', 'displayname': 'Pra...   
2  {'username': 'parmarmaninder', 'displayname': ...   
3  {'username': 'anmoldhaliwal', 'displayname': '...   
4  {'username': 'KotiaPreet', 'displayname': 'Pre...   

                                            outlinks  \
0  [https://twitter.com/ravisinghka/status/136415...   
1                                                 []   
2                                                 []   
3                     [https://youtu.be/-bUKumwq-J8]   
4                                                 []   

                 tcooutlinks  replyCount  retweetCount  ...  quoteCount  \
0  [https://t.co/es3kn0IQAF]           0             0  ...           0   
1                         []           0             0  ...           0   
2                         []           0             0  ...           0   
3  [https://t.co/wBPNdJdB0n]           0             0  ...           0   
4                         []           0             0  ...           0   

        conversationId  lang  \
0  1364506249291784198    en   
1  1364506237451313155    en   
2  1364506195453767680    pa   
3  1364350947099484160    en   
4  1364506144002088963   und   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/android" ...   
2  <a href="http://twitter.com/download/android" ...   
3  <a href="https://mobile.twitter.com" rel="nofo...   
4  <a href="http://twitter.com/download/iphone" r...   

                             sourceUrl          sourceLabel  \
0   http://twitter.com/download/iphone   Twitter for iPhone   
1  http://twitter.com/download/android  Twitter for Android   
2  http://twitter.com/download/android  Twitter for Android   
3           https://mobile.twitter.com      Twitter Web App   
4   http://twitter.com/download/iphone   Twitter for iPhone   

                                               media retweetedTweet  \
0                                               None            NaN   
1  [{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...            NaN   
2                                               None            NaN   
3  [{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...            NaN   
4  [{'previewUrl': 'https://pbs.twimg.com/media/E...            NaN   

                                         quotedTweet  \
0  {'url': 'https://twitter.com/RaviSinghKA/statu...   
1                                               None   
2                                               None   
3                                       

## Carga dataframe en GCP

### Configuración de carga

In [6]:
from google.cloud import bigquery

# Parámetros
project_id = 'project-latam-challenge'
dataset_id = 'twitter_data'
table_id = 'farmers_protest_tweets_2021'

# Inicializa el cliente de BigQuery
client = bigquery.Client(project=project_id)

# Referencia a la tabla
table_ref = client.dataset(dataset_id).table(table_id)


In [7]:
# Configuración del trabajo de carga
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Esto reemplaza la tabla si ya existe
)

# Cargar el DataFrame a BigQuery
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

# Esperar a que el trabajo de carga termine
job.result()

print(f"Cargado {job.output_rows} filas a {dataset_id}.{table_id}")


d:\valen\Documents\GitHub\latam-challenge\.venv\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: user, media, quotedTweet, mentionedUsers.
  warnings.warn(


Cargado 117407 filas a twitter_data.farmers_protest_tweets_2021
